In [ ]:
import MDAnalysis as mda
import MDAnalysis.transformations as trans
import numpy as np
side_atom = ["C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", 
             "C21", "C22", "C23", "C24", "C25", "C26", "C27", "C28"]

# upwrap structures and select backbone molecules
side_atom_selection = " or ".join([f"name {atom}" for atom in side_atom])
u = mda.Universe(f'../BULK_MD/P-SO2/BULK_GENERATION/06_Cooling300K/cooling06.tpr', f'../BULK_MD/P-SO2/BULK_GENERATION/06_Cooling300K/cooling06.gro')
u.trajectory.add_transformations(trans.unwrap(u.atoms))
molecule = u.select_atoms(f'name C* O* S* and not (resname PT* and ({side_atom_selection})) and not (resname PT* and name O*) and not (resname P2O and name O1 O2)')

molecule.write("../backbone.gro")
print(f"Selected {len(molecule)} atoms.")


## P2O
DBZ = u.select_atoms('(resname PT* and name C35 C36 C37 C38 C39 C40 C41 C42 C43 C44 C45 C46 S5)')
DBZ_2O = u.select_atoms('resname P2O and name C1 C2 C3 C4 C5 C6 C7 C8 C9 C10 C25 C26')
PTO = u.select_atoms(f'resname PT* and name C13 C14 C15 C16 C17 C18 C31 C32 C33 C34 S4 S2')
BEZ1 = u.select_atoms(f'resname P2O and name C11 C12 C13 C14 C23 C24') #接DBZ
BEZ2 = u.select_atoms(f'resname P2O and name C17 C18 C19 C20 C21 C22')

## PSO2
DBZ = u.select_atoms('(resname PT* and name C35 C36 C37 C38 C39 C40 C41 C42 C43 C44 C45 C46 S5)')
DBZ_2O = u.select_atoms('resname SO2 and name C7 C8 C9 C10 C11 C12 C13 C14 C15 C16 C17 C18 S3')
PTO = u.select_atoms(f'resname PT* and name C13 C14 C15 C16 C17 C18 C31 C32 C33 C34 S4 S2')
BEZ1 = u.select_atoms(f'resname SO2 and name C3 C4 C5 C6 C19 C20') #接DBZ
BEZ2 = u.select_atoms(f'resname SO2 and name C21 C22 C23 C24 C25 C26')

In [ ]:
output_file = "PSO2-pipi" # output file name

# read structure file
u = mda.Universe(f'../backbone.gro')
DBZ = u.select_atoms('(resname PT* and name C35 C36 C37 C38 C39 C40 C41 C42 C43 C44 C45 C46 S5)')
DBZ_2O = u.select_atoms('resname SO2 and name C7 C8 C9 C10 C11 C12 C13 C14 C15 C16 C17 C18 S3')
PTO = u.select_atoms(f'resname PT* and name C13 C14 C15 C16 C17 C18 C31 C32 C33 C34 S4 S2')
BEZ1 = u.select_atoms(f'resname SO2 and name C3 C4 C5 C6 C19 C20')
BEZ2 = u.select_atoms(f'resname SO2 and name C21 C22 C23 C24 C25 C26')

#cutoff range to see how parallel are normals to the plane of each monomer
parallelrange = 0.015
#cutoff radius for finding possbible CENTROID neighbors
cutoff = 15 ##Anstom
#cutoff distance for pistacks CENTROID in vertical (normal to their plane) and horizental (parallel to
verticalcutoff = 5 ##Anstom
horizentalcutoff = 5
#index.ndx is the index file for atoms exist in conjugated structure 
#that one wants to consider/total atoms in index file should be devisible by "number"
half_box= u.dimensions[1]/2 #Anstom

#calculate center of mass of each fragment
def COM(mol):
    com = mol.center_of_mass(compound='residues')
    return com

DBZ_com = COM(DBZ)
DBZ_2O_com = COM(DBZ_2O)
PTO_com = COM(PTO)
BEZ1_com = COM(BEZ1)
BEZ2_com = COM(BEZ2)

# calculate fragment normal vector
def com_vec(mol, com):
    a1= mol.atoms[0].position
    a2= mol.atoms[1].position

    v1 = a1-com
    v2 = a2-com
    normal_vector = np.cross(v1, v2)/np.linalg.norm(np.cross(v1,v2))
    return normal_vector

DBZ_vec = com_vec(DBZ, DBZ_com)
DBZ_2O_vec = com_vec(DBZ_2O, DBZ_2O_com)
PTO_vec = com_vec(PTO, PTO_com)
BEZ1_vec = com_vec(BEZ1, BEZ1_com)
BEZ2_vec = com_vec(BEZ2, BEZ2_com)

# Save information for each fragment into COM_DICT. This includes the center of mass (COM) and normal vectors.
COM_DICT = [
    list(zip(DBZ_com, DBZ_vec)),  
    list(zip(DBZ_2O_com, DBZ_2O_vec)), 
    list(zip(PTO_com, PTO_vec)), 
    list(zip(BEZ1_com, BEZ1_vec)),
    list(zip(BEZ2_com, BEZ2_vec))
]

In [ ]:
# Check for parallel fragments in COM_DICT based on their normal vectors.
def check_parallel(COM_DICT, parallelrange):
    PARALLEL = []
    for frag_idx1, fragment1 in enumerate(COM_DICT):
        # Check parallelism within the same fragment
        for i in range(len(fragment1)):
            for j in range(i + 1, len(fragment1)):
                com1, normal1 = fragment1[i]
                com2, normal2 = fragment1[j]
                cos_similarity = np.dot(normal1, normal2) / (np.linalg.norm(normal1) * np.linalg.norm(normal2))
                if (1.0 - parallelrange) < abs(cos_similarity):
                    PARALLEL.append((frag_idx1, i, frag_idx1, j))
        # Check parallelism between different fragments
        for frag_idx2, fragment2 in enumerate(COM_DICT):
            if frag_idx1 >= frag_idx2:
                continue
            for i in range(len(fragment1)):
                for j in range(len(fragment2)):
                    com1, normal1 = fragment1[i]
                    com2, normal2 = fragment2[j]
                    cos_similarity = np.dot(normal1, normal2) / (np.linalg.norm(normal1) * np.linalg.norm(normal2))
                    if (1.0 - parallelrange) < abs(cos_similarity):
                        PARALLEL.append((frag_idx1, i, frag_idx2, j))

    return PARALLEL
# Find parallel fragment pairs. The output is a list of tuples in the format:
# [(residue1_index, fragment1_index, residue2_index, fragment2_index)]
parrel_pairs = check_parallel(COM_DICT, parallelrange)

In [ ]:
# Check if pi-pi stacking pairs are within the cutoff distance
def NEIGHBOR(parrel_pairs, COM_DICT):
    neighbor = []
    for pair in parrel_pairs:
        residue_1 = pair[0] 
        residue_2 = pair[2] 
        frag_1 = pair[1]
        frag_2 = pair[3]
        com1 = COM_DICT[residue_1][frag_1][0] 
        com2 = COM_DICT[residue_2][frag_2][0]
        dist_vector = com2 - com1
        dist_vector -= np.round(dist_vector / (2 * half_box)) * (2 * half_box) 
        distance = np.linalg.norm(dist_vector)

        if distance < cutoff:
            neighbor.append(pair)  
    return neighbor

neighbor_pairs = NEIGHBOR(parrel_pairs, COM_DICT)


In [ ]:
# Check horizontal and vertical distances of pairs
def HOR_VIR (neighbor_pairs, COM_DICT):
    hor_vir = []
    hor_dis = []
    ver_dis = []
    for pair in neighbor_pairs:
        residue1 = pair[0]
        residue2 = pair[2]
        frag_1 = pair[1]
        frag_2 = pair[3]
        com1, vec1 = COM_DICT[residue1][frag_1][0], COM_DICT[residue1][frag_1][1]
        com2, vec2 = COM_DICT[residue2][frag_2][0], COM_DICT[residue2][frag_2][1]
        vector1 = np.subtract(com1, com2)
        vector1 -= np.round(vector1/(2*half_box))*(2*half_box)
        vector2 = vec2
        unit_vector1 = vector1/np.linalg.norm(vector1)
        unit_vector2 = vector2/np.linalg.norm(vector2)
        dot_product = np.dot(unit_vector1, unit_vector2)
        sine = np.sqrt(1-dot_product**2)
        d_vertical = abs(dot_product)*np.linalg.norm(vector1)
        d_horizon = sine*np.linalg.norm(vector1)

        if d_vertical < verticalcutoff and d_horizon < horizentalcutoff:
            hor_vir.append(pair)
            ver_dis.append(d_vertical)
            hor_dis.append(d_horizon)
    return hor_vir, ver_dis, hor_dis

hor_vir_pairs, vertical_dis, horizental_dis = HOR_VIR(neighbor_pairs, COM_DICT)

In [ ]:
AA = []
BB = []
CC = []
AB = []
AC = []
BC = []

for pair in hor_vir_pairs:
    if (pair[0] == 0 or pair[0] == 1) and (pair[2] == 0 or pair[2] == 1):
        AA.append(pair)
    elif pair[0] == 2 and pair[2] == 2:
        BB.append(pair)
    elif (pair[0] == 3 or pair[0] == 4) and (pair[2] == 3 or pair[2] == 4):
        CC.append(pair)
    elif (pair[0] == 0 or pair[0] == 1) and pair[2] == 2:
        AB.append(pair)
    elif (pair[0] == 0 or pair[0] == 1) and (pair[2] == 3 or pair[2] == 4):
        AC.append(pair)
    elif pair[0] == 2 and (pair[2] == 3 or pair[2] == 4):
        BC.append(pair)

with open(f"../{output_file}.txt" , 'w') as file:
    file.write(f"{len(hor_vir_pairs)}\n")
    file.write(f"AA: {len(AA)}\n")
    file.write(f"BB: {len(BB)}\n")
    file.write(f"CC: {len(CC)}\n")
    file.write(f"AB: {len(AB)}\n")
    file.write(f"AC: {len(AC)}\n")
    file.write(f"BC: {len(BC)}\n\n")
    file.write(f"Ver : {vertical_dis}\n")
    file.write(f"Hor : {horizental_dis}")

In [ ]:
name = {0: DBZ, 1: DBZ_2O, 2: PTO, 3: BEZ1, 4: BEZ2}

selected_atoms = u.select_atoms('') 

for pair in hor_vir_pairs:
    residue1 = pair[0]
    residue2 = pair[2]
    frag_1 = pair[1]
    frag_2 = pair[3]
    residue_atoms_1 = name[residue1] & name[residue1].residues[frag_1].atoms
    residue_atoms_2 = name[residue2] & name[residue2].residues[frag_2].atoms
    selected_atoms += residue_atoms_1 + residue_atoms_2
    
selected_atoms.write(f"../{output_file}.gro")

In [ ]:
# Load the complete structure from the GRO file and TPR file
u = mda.Universe(f'../BULK_MD/P-SO2/BULK_GENERATION/06_Cooling300K/cooling06.tpr', f'../BULK_MD/P-SO2/BULK_GENERATION/06_Cooling300K/cooling06.gro')
DBZ = u.select_atoms('(resname PT* and name C35 C36 C37 C38 C39 C40 C41 C42 C43 C44 C45 C46 S5)')
DBZ_2O = u.select_atoms('resname SO2 and name C7 C8 C9 C10 C11 C12 C13 C14 C15 C16 C17 C18 S3')
PTO = u.select_atoms(f'resname PT* and name C13 C14 C15 C16 C17 C18 C31 C32 C33 C34 S4 S2')
BEZ1 = u.select_atoms(f'resname SO2 and name C3 C4 C5 C6 C19 C20')
BEZ2 = u.select_atoms(f'resname SO2 and name C21 C22 C23 C24 C25 C26')
name = {0: DBZ, 1: DBZ_2O, 2: PTO, 3: BEZ1, 4: BEZ2}

In [ ]:
categories = ["AA", "BB", "CC", "AB", "AC", "BC"]
category_counts = {cat: {"same": 0, "not same": 0} for cat in categories}
selected_atoms = u.select_atoms('') 
# classify pi-pi stacking pairs
for pair in hor_vir_pairs:
    residue1 = pair[0]
    residue2 = pair[2]
    frag_1 = pair[1]
    frag_2 = pair[3]
    atoms1 = name[residue1].residues[frag_1].atoms[0]
    atoms2 = name[residue2].residues[frag_2].atoms[0]
    fragment1 = atoms1.fragment
    fragment2 = atoms2.fragment
    # decide pi-pi stacking type
    if (residue1 in [0, 1] and residue2 in [0, 1]):
        category = "AA"
    elif residue1 == 2 and residue2 == 2:
        category = "BB"
    elif (residue1 in [3, 4] and residue2 in [3, 4]):
        category = "CC"
    elif (residue1 in [0, 1] and residue2 == 2):
        category = "AB"
    elif (residue1 in [0, 1] and residue2 in [3, 4]):
        category = "AC"
    elif (residue1 == 2 and residue2 in [3, 4]):
        category = "BC"
    else:
        continue
    
    # Determine whether pi-pi stacking pairs are intrachain or interchain
    if fragment1 == fragment2:
        category_counts[category]["same"] += 1
    else:
        category_counts[category]["not same"] += 1

for cat in categories:
    print(f"{cat}: [{category_counts[cat]['same']:>3},{category_counts[cat]['not same']:>3}]")

with open(f"../{output_file}.txt" , 'a') as file:
    for cat in categories:
        file.write(f"{cat}: [{category_counts[cat]['same']:>3},{category_counts[cat]['not same']:>3}]\n")

